##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ML Metadata による ML エンジニアリングの改善


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/mlmd/mlmd_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/mlmd/mlmd_tutorial.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
  
</table>

ペンギンを分類する本番 ML パイプラインをセットアップしたと仮定します。トレーニングデータを取り込んでモデルのトレーニングと評価を行い、本番にプッシュするパイプラインです。

ところが後で、様々な種類のペンギンを含むより大きなデータセットでこのモデルを使用してみると、モデルが期待したように動作せず、種を誤って分類し始めてしまいました。

この時点で、以下について知ろうとするでしょう。

- 唯一利用できるアーティファクトが本番のモデルである場合、そのモデルを最も効率的にデバッグする方法は？
- モデルのトレーニングには使用されたデータセットは？
- 不具合のあるモデルを生じたトレーニングランは？
- モデルの評価結果の場所は？
- デバッグを始める場所は？

[ML Metadata（MLMD）](https://github.com/google/ml-metadata)は、ML モデルに関連付けられたメタデータを使用して、上記やそれ以外の疑問に対する答えを見つけられるようにするライブラリです。分かりやすく例えると、このメタデータは、ソフトウェア開発におけるロギングに相当すると考えられるでしょう。MLMD では、ML パイプラインの様々なコンポーネントに関連付けられたアーティファクトとリネージを確実に追跡することができます。

このチュートリアルでは、TFX パイプラインをセットアップして、ペンギンを、体重、嘴峰（しほう）長、嘴峰高、フリッパーの長さに基づいて 3 つの種に分類するモデルを作成します。次に、MLMD を使用して、パイプラインコンポーネントのリネージを追跡します。

## Colab における TFX パイプライン

Colab は、本番環境とは大きく異なる軽量の開発環境です。本番では、データ取り込み、変換、モデルトレーニング、ラン履歴など、複数の分散システムをまたぐ様々なパイプラインコンポーネントが伴うことがあります。このチュートリアルでは、オーケストレーションとメタデータストレージに大きな違いがあり、すべて Colab 内でローカルに処理されることに注意しておく必要があります。Colab での TFX についての詳細は、[こちら](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#background)をご覧ください。


## MNIST モデルをビルドする

まず、必要なパッケージをインストールしてインポートし、パスを設定して、データをダウンロードします。

### Pip のアップグレード

ローカルで実行する場合にシステム Pip をアップグレードしないようにするには、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップグレードできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX をインストールしてインポートする

In [ ]:
 !pip install -q tfx

### パッケージをインポートする

#### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「ランタイムを再起動」ボタンをクリックするか、「ランタイム」&gt;「ランタイムの再起動...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

In [ ]:
import os
import tempfile
import urllib
import pandas as pd

import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

TFX と MLMD のバージョンを確認します。

In [ ]:
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import ml_metadata as mlmd
print('MLMD version: {}'.format(mlmd.__version__))

## データセットをダウンロードする

この Colab では、[Github](https://github.com/allisonhorst/palmerpenguins) にある [Palmer Penguins データセット](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)を使用します。このデータセットは、不完全なレコードを除外し、`island` と `sex` 列を削除し、ラベルを `int32` に変換して処理されています。データセットには、ペンギンの体重、嘴峰長、嘴峰高、フリッパーの長さの 334 件のレコードが含まれています。このデータを使用して、ペンギンを 3 つの種のいずれかに分類します。

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "penguins_processed.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

## InteractiveContext を作成する

TFX コンポーネントをこのノートブックで対話的に実行するには、`InteractiveContext` を作成します。`InteractiveContext` は、エフェメラル MLMD データベースインスタンスのある一時ディレクトリを使用します。`InteractiveContext` への呼び出しは、Colab 環境外では no-ops であることに注意してください。

一般に、類似するパイプラインランを 1 つの `Context` にグループ化することが推奨されています。

In [ ]:
interactive_context = InteractiveContext()

## TFX パイプラインを構築する

TFX パイプラインは、ML ワークフローの様々な側面を実行する複数のコンポーネントで構成されます。このノートブックでは、 `ExampleGen`、`StatisticsGen`、`SchemaGen`、および `Trainer` コンポーネントを作成して実行し、`Evaluator` と `Pusher` コンポーネントによって、トレーニングされたモデルの評価とプッシュを行います。

TFX パイプラインのコンポーネントについての詳細は、[コンポーネントのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)をご覧ください。

注意: 個別のコンポーネントをセットアップして TFX パイプラインを構築するには、大量のボイラープレートコードが伴います。このチュートリアルの目的においては、パイプラインセットアップのすべてのコードを完全に理解しなくても構いません。 

### ExampleGen コンポーネントをインスタンス化して実行する

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
interactive_context.run(example_gen)

### StatisticsGen コンポーネントをインスタンス化して実行する

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

### SchemaGen コンポーネントをインスタンス化して実行する

In [ ]:
infer_schema = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

### Trainer コンポーネントをインスタンス化して実行する


In [ ]:
# Define the module file for the Trainer component
trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_LABEL_KEY = 'species'

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema, batch_size: int) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY), schema).repeat()


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: tfx.components.FnArgs):
  schema = schema_pb2.Schema()
  tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema)
  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=10)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

`Trainer` コンポーネントを実行します。

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
interactive_context.run(trainer)

### モデルを評価してプッシュする

`Evaluator` コンポーネントを使用してモデルの評価と「ブレッシング」を行い、`Pusher` コンポーネントを使用してモデルをサービングディレクトリにプッシュします。

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/penguins_classification')

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [ ]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

TFX パイプラインを実行すると、MLMD データベースにデータが入力されます。次のセクションでは、MLMD API によってこのデータベースをクエリし、メタデータ情報を取得します。

## MLMD データベースをクエリする

MLMD データベースは、3 種類のメタデータを格納します。

- パイプラインコンポーネントに関連付けられたパイプラインとリネージの情報に関するメタデータ
- パイプラインランで生成されたアーティファクトに関するメタデータ
- パイプラインの実行に関するメタデータ

典型的な本番環境パイプラインは、新しいデータが届くたびに複数のモデルを配信します。配信されたモデルにエラーのある結果が含まれる場合、MLMD データベースをクエリし、不具合のあるモデルを分離することができます。その上で、それらのモデルに対応するパイプラインコンポーネントのリネージを追跡し、モデルをデバッグすることができます。

前に定義した `InteractiveContext` を使用して、MLMD データベースをクエリするメタデータ（MD）ストアをセットアップします。

In [ ]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

MD ストアからデータを表示するためのヘルパー関数を作成します。

In [ ]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [ ]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [ ]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

まず、MD ストアに対し、格納されたすべての `ArtifactTypes` のリストをクエリします。

In [ ]:
display_types(store.get_artifact_types())

次に、すべての `PushedModel` アーティファクトをクエリします。

In [ ]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

MD ストアに対し、最後にプッシュされたモデルをクエリします。このチュートリアルには、プッシュされたモデルが 1 つしかありません。 

In [ ]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

プッシュされたモデルをデバッグする際の最初のステップは、どのモデルがプッシュされており、そのモデルをトレーニングする上でどのトレーニングデータが使用されたかを確認することです。

MLMD には、来歴グラフを走査するトラバーサル API があるため、それを使用して、モデルの来歴を分析することができます。 

In [ ]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

親アーティファクトに対し、プッシュされたモデルをクエリします。

In [ ]:
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

プロパティに対し、モデルをクエリします。

In [ ]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

上流アーティファクトに対し、モデルをクエリします。

In [ ]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

モデルのトレーニングに使用されたトレーニングデータを取得します。

In [ ]:
used_data = model_parents[0]
display_properties(store, used_data)

モデルのトレーニングに使用されたトレーニングデータを取得したので、もう一度データベースをクエリし、トレーニングステップ（実行）を見つけます。MD ストアに対し、登録された実行タイプのリストをクエリします。

In [ ]:
display_types(store.get_execution_types())

トレーニングステップは `tfx.components.trainer.component.Trainer` という `ExecutionType` です。MD ストアを走査して、プッシュされたモデルに対応する trainer ランを取得します。

In [ ]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == mlmd.proto.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

## 目次

このチュートリアルでは、MLMD を使用して、TFX パイプラインコンポーネントのリネージを追跡し、問題を解決する方法について学習しました。

MLMD の使用方法についての詳細は、以下の追加リソースをご覧ください。

- [MLMD API ドキュメント](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd)
- [MLMD ガイド](https://www.tensorflow.org/tfx/guide/mlmd)